In [64]:
url = r"C:\Users\princ\OneDrive\Documents\Chapter 1\Python\Machine Learning\ML Data\news.csv\news.csv"

In [93]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer


In [94]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\princ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [95]:
#Printing the stopwords in english
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [96]:
#uploading and converting the data into the pandas dataframe
df= pd.read_csv(url)
data = pd.DataFrame(df)
data

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [92]:
#checking for missing values
data.isnull().sum()

Unnamed: 0         0
title           6335
text               0
label              0
label_binary       0
dtype: int64

In [70]:
#Seeing the first five rows of the data
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [71]:
#changing the label column into binary columns using label encoder
le = LabelEncoder()
data['label_binary'] = le.fit_transform(data['label'])
data.head()

,Unnamed: 0,title,text,label,label_binary
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,1


It has converted the column into binary side denoting 0--Fake,1--Real


In [72]:
#Printing the shape of data
data.shape

(6335, 5)

In [73]:
data['label_binary'].value_counts()

label_binary
1    3171
0    3164
Name: count, dtype: int64

In [74]:
#Sepratinng the data and label
X = data.drop(columns = ['label','label_binary'],axis = 1)
y = data['label']
print(X)
print(y)

      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text  
0     Daniel Greenfield, a Shillman Journalism Fello...  
1     Google Pinterest Digg Lin

Stemming :
stemming is the process of reducing a word to its root word
example:
actor,actress,acting--> act

In [75]:
port_stem = PorterStemmer()

In [102]:
def stemming(content):
    stemmed_content = re.sub('[˄a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)   
    return stemmed_content  

  

In [103]:

def stemming(text):
    # Check if the text is None or NaN
    if pd.isna(text):
        return ""  # Return empty string for None/NaN values
    
    # Original stemming logic for valid text
    # (assuming your stemming function processes strings)
    # Replace this with your actual stemming implementation
    return text  # Placeholder - replace with actual stemming logic

# Apply the modified stemming function
data['text'] = data['text'].apply(stemming)


In [104]:
print(data['text'])

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: text, Length: 6335, dtype: object


In [105]:
#Seprating the data and label
X = data['text'].values
y = data['label'].values

In [106]:
print(X)

['Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership is being warned that the enti

In [107]:
print(y)

['FAKE' 'FAKE' 'REAL' ... 'FAKE' 'REAL' 'REAL']


In [108]:
y.shape

(6335,)

In [109]:
#Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [110]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2158282 stored elements and shape (6335, 67659)>
  Coords	Values
  (0, 1621)	0.016229531067969236
  (0, 2306)	0.021002759070217546
  (0, 2399)	0.01241685144719205
  (0, 2486)	0.024110528944251836
  (0, 2636)	0.016094233592590813
  (0, 2640)	0.043376791257086454
  (0, 2749)	0.05135437224587028
  (0, 2978)	0.023340051773888364
  (0, 3033)	0.02038313224006218
  (0, 3214)	0.06004278866572469
  (0, 3234)	0.007165842283234525
  (0, 3256)	0.03381419923018935
  (0, 3265)	0.01624011518858776
  (0, 3274)	0.01540403684145884
  (0, 3278)	0.03637152475123043
  (0, 3329)	0.011766645990727378
  (0, 3375)	0.014600249496516973
  (0, 3749)	0.020301409842698546
  (0, 3761)	0.03255183170443468
  (0, 3773)	0.019337300942003526
  (0, 3812)	0.030843112273660794
  (0, 3834)	0.015249463170958514
  (0, 3900)	0.02188364387038962
  (0, 3905)	0.013747932717198952
  (0, 4066)	0.018136295662544426
  :	:
  (6334, 64842)	0.0211113566867202
  (6334, 64851)	0

In [111]:
#Splitting the data for training and testing the model
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify = y,random_state = 2)

In [112]:
print(X.shape,X_train.shape,X_test.shape)

(6335, 67659) (5068, 67659) (1267, 67659)


In [113]:
model = LogisticRegression()
#Training the model on training data
model.fit(X_train,y_train)

LogisticRegression()

In [114]:
#Accuracy of model on the training data
X_train_pred = model.predict(X_train)
Training_data_accuracy = accuracy_score(X_train_pred,y_train)
print("Accuracy score on the training data is:",Training_data_accuracy) 

Accuracy score on the training data is: 0.9530386740331491


In [116]:
#Evaluating the model accuracy on the testing data
X_test_pred = model.predict(X_test)
Testing_data_accuracy = accuracy_score(X_test_pred,y_test)
print("Accuracy of model on Test data is: ",Testing_data_accuracy)

Accuracy of model on Test data is:  0.9123914759273876


Making a Predictive System

In [121]:
# The model expects 67659 features, but we only have 1 value
# Option 1: If this is a text classification model, you need to preprocess your text first
# For example, if using a vectorizer:

# Assuming you have your text data and the vectorizer used during training
text_data = "In a lot of elections, being the studious one would have worked, he said, but Trump has flipped the script."  # Replace with your actual news text
# First vectorize the text using the same vectorizer used during training
input_data_vectorized = vectorizer.transform([text_data])  # vectorizer should be the same one used for training

# Now predict with the properly vectorized data
prediction = model.predict(input_data_vectorized)

# Option 2: If you need to create a dummy input with correct dimensions for testing:
# import numpy as np
# input_data_dummy = np.zeros((1, 67659))  # Create array with correct number of features
# input_data_dummy[0, 0] = 0.021063723188870318  # Set your value at desired position
# prediction = model.predict(input_data_dummy)

if prediction[0] == 0:
    print("Fake News")
else:
    print("Real News")

Real News
